In [2]:
from collections import Counter, defaultdict
from enum import Enum
from typing import List, NamedTuple, Tuple, Dict, TypeVar, Callable, Iterable

In [3]:
def get_lines(day: int) -> List[str]:
    with open(f"inputs/day{day}.txt", "r") as f:
        return f.read().splitlines()

# --- Day 1: Sonar Sweep ---


In [4]:
def count_increases(depths: List[int]) -> int:
    """Counts the number of times the depth increases"""

    return sum(d2 - d1 > 0 for d1, d2 in zip(depths, depths[1:]))

In [5]:
def sliding_sum(depths: List[int], window_size: int = 3) -> List[int]:
    """Computes a window_size-element sliding sum of the list of numbers"""

    views: List[List[int]] = [depths[i:] for i in range(window_size)]
    return [sum(window) for window in zip(*views)]

In [6]:
lines: List[str] = get_lines(1)
depths: List[int] = [int(line) for line in lines]

print(f"Answer #1: {count_increases(depths)}")
print(f"Answer #2: {count_increases(sliding_sum(depths))}")

Answer #1: 1532
Answer #2: 1571


# --- Day 2: Dive! ---


In [7]:
class Direction(Enum):
    FORWARD = 1
    DOWN = 2
    UP = 3

In [8]:
class Command(NamedTuple):
    direction: Direction
    magnitude: int

In [9]:
def parse_command(command: str) -> Command:
    """Parses a command assuming the format"""
    direction_mapping = {"forward": Direction.FORWARD, "down": Direction.DOWN, "up": Direction.UP}
    raw_direction, raw_magnitude = command.split(" ")
    
    return Command(direction_mapping[raw_direction], int(raw_magnitude))

In [10]:
def position_times_depth(commands: List[Command], use_aim=False) -> int:
    horizontal_position = 0
    depth = 0
    aim = 0

    for command in commands:
        match command, use_aim:
            case Command(Direction.FORWARD, x), True:
                horizontal_position += x
                depth += x * aim
            case Command(Direction.FORWARD, x), False:
                horizontal_position += x
            case Command(Direction.DOWN, x), True:
                aim += x
            case Command(Direction.DOWN, x), False:
                depth += x
            case Command(Direction.UP, x), True:
                aim -= x
            case Command(Direction.UP, x), False:
                depth -= x
    
    return horizontal_position * depth

In [11]:
lines: List[str] = get_lines(2)
commands: List[Command] = [parse_command(line) for line in lines]

print(f"Answer #1 {position_times_depth(commands)}")
print(f"Answer #2 {position_times_depth(commands, use_aim=True)}")

Answer #1 2120749
Answer #2 2138382217


# --- Day 3: Binary Diagnostic ---

In [12]:
def parse_reading(line: str) -> List[int]:
    return [int(x) for x in list(line)]

In [13]:
def count_values(readings: List[List[int]]) -> Dict[Tuple[int, int], int]:
    counter = defaultdict(int)

    for reading in readings:
        for i, value in enumerate(reading):
            counter[(i, value)] += 1

    return counter


In [14]:
def get_power_consumption(readings: List[List[int]]) -> int:
    epsilon_binary = []
    gamma_binary = []

    counter = count_values(readings)

    for i in range(len(readings[0])):
        most_common_value = max(0, 1, key=lambda value: counter[(i, value)])
        least_common_value = int(not most_common_value)
        
        epsilon_binary.append(str(most_common_value))
        gamma_binary.append(str(least_common_value))

    epsilon = int("".join(epsilon_binary), 2)
    gamma = int("".join(gamma_binary), 2)

    return epsilon * gamma


In [15]:
def get_life_support_rating(readings: List[List[int]]) -> int:    
    oxygen_candidates = readings
    co2_candidates = readings
    oxygen_rating = None
    co2_rating = None

    for i in range(len(readings[0])):
        oxygen_counter = count_values(oxygen_candidates)
        co2_counter = count_values(co2_candidates)
        o2_count_0 = oxygen_counter[(i, 0)]
        o2_count_1 = oxygen_counter[(i, 1)]
        co2_count_0 = co2_counter[(i, 0)]
        co2_count_1 = co2_counter[(i, 1)]

        if o2_count_0 == o2_count_1:
            oxygen_predicate = lambda x: x == 1
        else:
            oxygen_predicate = lambda x: x == max(0, 1, key=lambda v: oxygen_counter[(i, v)])

        if co2_count_0 == co2_count_1:
            co2_predicate = lambda x: x == 0
        else:
            co2_predicate = lambda x: x == min(0, 1, key=lambda v: co2_counter[(i, v)])

        oxygen_candidates = [reading for reading in oxygen_candidates if oxygen_predicate(reading[i])]
        co2_candidates = [reading for reading in co2_candidates if co2_predicate(reading[i])]

        if len(oxygen_candidates) == 1:
            oxygen_rating = int("".join(str(x) for x in oxygen_candidates[0]), 2)
        
        if len(co2_candidates) == 1:
            co2_rating = int("".join(str(x) for x in co2_candidates[0]), 2)

        if oxygen_rating is not None and co2_rating is not None:
            break

    return oxygen_rating * co2_rating


In [16]:
lines: List[str] = get_lines(3)
readings: List[List[int]] = [parse_reading(line) for line in lines]

print(f"Answer #1: {get_power_consumption(readings)}")
print(f"Answer #2: {get_life_support_rating(readings)}")

Answer #1: 3374136
Answer #2: 4432698


# --- Day 4: Giant Squid ---

In [17]:
Board = List[List[Tuple[int, bool]]]

In [18]:
def parse_nums(raw_nums: str) -> List[int]:
    return [int(num) for num in raw_nums.split(",")]

In [19]:
def parse_boards(raw_boards: List[str]) -> List[Board]:
    return [[[(int(x), False) for x in row.split(" ") if x] for row in board.split("\n") if row] for board in raw_boards]

In [20]:
def rows_solved(board: Board) -> bool:
    for row in board:
        if all(marked for marked in row):
            return True
    return False

In [21]:
def board_solved(board: Board) -> bool:
    marked_board = [[marked for _, marked in row] for row in board]
    transposed_board = list(map(list, zip(*marked_board)))
    return rows_solved(marked_board) or rows_solved(transposed_board)

In [22]:
def update_board(board: Board, drawn_num: int) -> Board:
    updated_board = []
    for row in board:
        updated_row = []
        for num, marked in row:
            updated_row.append((num, drawn_num == num or marked))
        updated_board.append(updated_row)
    
    return updated_board

In [23]:
def compute_bingo_score(board: Board, num: int) -> int:
    unmarked_sum = sum(x for row in board for x, marked in row if not marked)

    return unmarked_sum * num

In [24]:
def solve_bingo(drawn_nums: List[int], boards: List[Board], winner_first=True) -> int:
    updated_boards = boards
    solved_boards = []
    for num in drawn_nums:
        updated_boards = [update_board(board, num) for board in updated_boards if board not in solved_boards]
        for board in updated_boards:
            if board_solved(board):
                if winner_first:
                    return compute_bingo_score(board, num)
                solved_boards.append(board)
                if len(solved_boards) == len(boards):
                    return compute_bingo_score(board, num)

In [25]:
with open("inputs/day4.txt", "r") as f:
    raw_nums, *raw_boards = f.read().split("\n\n")
    
    nums = parse_nums(raw_nums)
    boards = parse_boards(raw_boards)

print(f"Answer #1: {solve_bingo(nums, boards)}")
print(f"Answer #2: {solve_bingo(nums, boards, winner_first=False)}")

Answer #1: 35670
Answer #2: 22704


# --- Day 5: Hydrothermal Venture ---

In [26]:
Point = Tuple[int, int]
Segment = Tuple[Point, Point]

In [27]:
def point_slope(x: int, segment: Segment) -> Point:
    ((x1, y1), (x2, y2)) = segment
    y = (y2 - y1) / (x2 - x1) * (x - x1) + y1
    
    return x, int(y)

In [28]:
def parse_segments(lines: List[str]) -> List[Segment]:
    return [[tuple(int(coord) for coord in raw_point.split(",")) for raw_point in line.split(" -> ")] for line in lines]

In [29]:
def unfold_segment(segment: Segment, count_diagonals=True) -> List[Point]:
    ((x1, y1), (x2, y2)) = segment
    min_x, max_x = sorted([x1, x2])
    min_y, max_y = sorted([y1, y2])
    domain = range(min_x, max_x + 1)

    if x1 == x2:
        return [(x1, y) for y in range(min_y, max_y + 1)]

    return [point_slope(x, segment) for x in domain if count_diagonals or y1 == y2]

In [30]:
def unfold_points(segments: List[Segment], count_diagonals=True) -> List[Point]:
    return [point for segment in segments for point in unfold_segment(segment, count_diagonals=count_diagonals)]

In [31]:
def count_overlapping_points(points: List[Point], threshold: int = 2) -> int:
    counter = Counter(points)

    return sum(1 for _, count in counter.items() if count >= threshold)

In [32]:
with open("inputs/day5.txt", "r") as f:
    lines = f.read().splitlines()
    segments = parse_segments(lines)
    straight_points = unfold_points(segments, count_diagonals=False)
    points = unfold_points(segments)

print(f"Answer #1: {count_overlapping_points(straight_points)}")
print(f"Answer #1: {count_overlapping_points(points)}")

Answer #1: 5608
Answer #1: 20299


# --- Day 6: Lanternfish ---

In [33]:
def simulate_lanternfish_naive(fish: List[int], days=80) -> int:
    fish_counter = fish.copy()
    for day in range(days):
        new_fish_counter = []
        for fish_timer in fish_counter:
            if fish_timer == 0:
                new_fish_counter.append(6)
                new_fish_counter.append(8)
            else:
                new_fish_counter.append(fish_timer - 1)
        fish_counter = new_fish_counter

    return len(fish_counter)

In [34]:
def simulate_lanternfish(fish: List[int], days=256) -> int:
    fish_counter = Counter(fish)

    for _ in range(days):
        spawned = fish_counter.pop(0, 0)
        fish_counter = Counter({i - 1: fish for i, fish in fish_counter.items()})
        fish_counter[8] += spawned
        fish_counter[6] += spawned


    return sum(fish_counter.values())

In [35]:
with open("inputs/day6.txt", "r") as f:
    fish = [int(fish) for fish in f.read().split(",")]

print(f"Answer #1 {simulate_lanternfish_naive(fish)}")
print(f"Answer #2 {simulate_lanternfish(fish, days=256)}")

Answer #1 374927
Answer #2 1687617803407


# --- Day 7: The Treachery of Whales ---


In [36]:
def cheapest_rendezvous(crabs: List[int], nonlinear_fuel=False) -> int:
    positions = range(min(crabs), max(crabs))
    fuel_counts = Counter()

    for crab in crabs:
        for position in positions:
            distance = abs(position - crab)
            if nonlinear_fuel:
                fuel = int((distance * (distance + 1)) / 2) 
            else:
                fuel = distance 
            fuel_counts[position] += fuel

    return fuel_counts.most_common()[-1][1]

In [37]:
with open("inputs/day7.txt", "r") as f:
    crabs = [int(crabs) for crabs in f.read().split(",")]

print(f"Answer #1: {cheapest_rendezvous(crabs)}")
print(f"Answer #2: {cheapest_rendezvous(crabs, nonlinear_fuel=True)}")

Answer #1: 326132
Answer #2: 88612508


# --- Day 8: Seven Segment Search ---

In [38]:
def parse_patterns(line: str) -> Tuple[List[str], List[str]]:
    raw_patterns, raw_outputs = line.split("|")
    patterns = raw_patterns.split(" ")
    outputs = raw_outputs.split(" ")

    return patterns, outputs

In [39]:
def find_pattern_with_length(pattern: List[str], length: int) -> List[str]:
    for pattern in pattern:
        if len(pattern) == length:
            return pattern

In [40]:
def unique_digit_appearance(patterns: List[str], outputs: List[str]) -> int:
    digit_length_mapping = {1: 2, 4: 4, 7: 3, 8: 7}

    return sum(1 for output in outputs if len(output) in digit_length_mapping.values())


In [41]:
A = TypeVar("A")
B = TypeVar("B")
C = TypeVar("C")

In [42]:
def parse_and_reduce(lines: List[str], parser: Callable[[str], A], mapper: Callable[[A], B], reducer: Callable[[Iterable[B]], C], star_args=False) -> C:
    parsed = [parser(line) for line in lines]

    return reducer(mapper(*p) if star_args else mapper(p) for p in parsed)

In [43]:
def unique_digit_appearances(lines: List[str]) -> int:
    return parse_and_reduce(lines, parse_patterns, unique_digit_appearance, sum, star_args=True)

In [44]:
def decode_and_sum_output(patterns: List[str], outputs: List[str]) -> int:
    digit_length_mapping = {1: 2, 4: 4, 7: 3, 8: 7}
    length_digit_mapping = {v: k for k, v in digit_length_mapping.items()}
    pattern_map = {digit: find_pattern_with_length(patterns, length) for digit, length in digit_length_mapping.items()}

    # 1: ab
    # 4: eafb
    # 7: dab
    # 8: acedgfb

    


In [45]:
def decode_and_sum_outputs(lines: List[str]) -> int:
    return parse_and_reduce(lines, parse_patterns, decode_and_sum_output, sum, star_args=True)

In [46]:
with open("inputs/day8.txt", "r") as f:
    lines = f.read().splitlines()

print(f"Answer #1: {unique_digit_appearances(lines)}")
print(f"Answer #2: {decode_and_sum_outputs(lines)}")

Answer #1: 532


TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'